In [3]:
import sys
sys.executable

'/Users/sitewang/.pyenv/versions/3.11.9/envs/gtcsummary/bin/python'

In [4]:
import os
import PyPDF2

In [5]:
def extract_text_from_first_page(pdf_path):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        first_page = pdf_reader.pages[0]
        text = first_page.extract_text()
        return text.strip() if text else 'Text Not Found'

def extract_texts_from_directory(directory_path):
    texts = {}
    for filename in os.listdir(directory_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(directory_path, filename)
            text = extract_text_from_first_page(pdf_path)
            texts[filename] = text
    return texts

# Specify the directory containing the PDFs
pdf_directory = '/Users/sitewang/Desktop/talks/'
texts = extract_texts_from_directory(pdf_directory)

for filename, title in texts.items():
    print(f'Filename: {filename}, Title: {title}')
    break

FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-10') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead
FloatObject (b'0.00-20') invalid; use 0.0 instead


Filename: S62568_1711141735236001lMUE.pdf, Title: Edge Computing 101
An Introduction to the Smart Edge
March , 24
Chen Su, Sr Technical Product Marketing Manager


In [25]:
import ollama
import re
import time
from jinja2 import Template
import google.generativeai as genai

In [8]:
print(f"Total {len(texts)} files")

Total 139 files


In [9]:
slide_titles = {}

In [26]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))
model = genai.GenerativeModel('gemini-pro')

In [28]:
prompt_template = """
Given the extracted text from the first slide of a keynote (Note that the text can be disorderd):
'Accelerating NetworkX: \nThe Future of Easy Graph Analytics \nMridul Seth - Core Developer, 
NetworkX \nErik Welch - Sr. System Software Engineer, NVIDIA \nRick Ratzel - Sr. System Software 
Engineer, NVIDIA \nNetworkX and backend developers'

Restore the title of the this keynote in json format is:
{'title': 'Accelerating NetworkX: The Future of Easy Graph Analytics by Mirdul Seth and Erik Welch from NetworkX and Erik Welch, Rick Ratzel from NVIDIA'}

Don't use anything word outside the extracted text. Leave anything that does not belong to the title out.

Using the same logic, below is text extracted from the another slide of a keynote:
{{ title_slide }}

The restored title of this keynote in json format is:
"""

for filename, title in texts.items():
    if filename in slide_titles:
        continue
    # messages = [
    #     {"role": "system", "content": "You are a helpful agent who help summarize and extract information."},
    #     {"role": "user", "content": Template(prompt_template).render({'title_slide': title})}
    # ]
    
    response = model.generate_content(Template(prompt_template).render({'title_slide': title}))
    # response = ollama.chat(model='mistral', messages=messages)
    print(response.text)
    
    try: 
        # cleaned_title = eval(response['message']['content'])
        cleaned_title = eval(response.text)
    except:
        cleaned_title = {"title": ""}
        print(f"Error with {filename}")
    
    time.sleep(1)
    
    slide_titles[filename] = {
        "title_text": title,
        "cleaned_title": cleaned_title.get('title', '')
    }

{'title': 'NetworkX and backend developers'}
{'title': 'Tencent Text-to-Image Generative Model by Lu Qinglin'}
{'title': 'How Genius Sports transforms NFL game viewing with accelerated computing on AWS by Shruti Koparkar and Sheldon Kwok'}
{'title': 'A Blueprint for LLM Cluster Architecture'}
{'title': 'Harnessing Generative AI and Language Models With Vision AI AgentsPadmavathy Subramanian, Kaustubh Purandare, Subashree Radhakrishnan'}
{'title': 'ControlExpert+ NVIDIAInsurance Claims in the Age of AIAI-Powered Insurance Claim'}
{'title': 'Recommender Systems 101: Accelerated Training atScale by Minseok Lee'}
{'title': 'Robotics in the age of Generative AI by Vincent Vanhoucke from Google DeepMind'}
{'title': 'Accelerating and Securing GPU Accesses to Large Datasets by CJ Newburn, Oren Duer, and Vikram Mailthody'}
{'title': 'Digital Twins for Immersive Location-Based Entertainment by Moment Factory Innovation at NVIDIA GTC, March 2024'}
{'title': 'GTC 2024Kawasaki Track Maintenance Pla

In [29]:
slide_titles

{'S62568_1711141735236001lMUE.pdf': {'title_text': 'Edge Computing 101\nAn Introduction to the Smart Edge\nMarch , 24\nChen Su, Sr Technical Product Marketing Manager',
  'cleaned_title': 'Edge Computing 101: An Introduction to the Smart Edge by Chen Su'},
 'S62359_1708641953650001ogDb.pdf': {'title_text': 'Machine Learning has Taken Weather Forecasting by Storm How About Climate Modeling?\nChristopher Bretherton Senior Director of Climate Modeling Allen Institute for Artificial Intelligence (AI2), Seattle, USACA atmospheric riverFeb. 4, 2024NASA Worldview',
  'cleaned_title': 'Machine Learning has Taken Weather Forecasting by Storm: How About Climate Modeling? by Christopher Bretherton from Allen Institute for Artificial Intelligence (AI2)'},
 'S62192_1711057720932001KuIu.pdf': {'title_text': '1\nAdvanced Performance Optimization in CUDA\n[S62192]\nIgor Terentyev*, NVIDIA DevTech Compute\nGPU Technology Conference  /  March 18th, 2024\n* With Guillaume Thomas -Collignon & Athena Elafr

In [40]:
for k, v in slide_titles.items():
    print(k, v)
    print(f"{k} || {v.get('cleaned_title', '')}")

S62568_1711141735236001lMUE.pdf {'title_text': 'Edge Computing 101\nAn Introduction to the Smart Edge\nMarch , 24\nChen Su, Sr Technical Product Marketing Manager', 'cleaned_title': 'Edge Computing 101: An Introduction to the Smart Edge by Chen Su'}
S62568_1711141735236001lMUE.pdf || Edge Computing 101: An Introduction to the Smart Edge by Chen Su
S62359_1708641953650001ogDb.pdf {'title_text': 'Machine Learning has Taken Weather Forecasting by Storm How About Climate Modeling?\nChristopher Bretherton Senior Director of Climate Modeling Allen Institute for Artificial Intelligence (AI2), Seattle, USACA atmospheric riverFeb. 4, 2024NASA Worldview', 'cleaned_title': 'Machine Learning has Taken Weather Forecasting by Storm: How About Climate Modeling? by Christopher Bretherton from Allen Institute for Artificial Intelligence (AI2)'}
S62359_1708641953650001ogDb.pdf || Machine Learning has Taken Weather Forecasting by Storm: How About Climate Modeling? by Christopher Bretherton from Allen Ins

In [31]:
# Manual Review Section
target = 'S62613_1710772813836001Zn9R.pdf'
for filename, title in texts.items():
    if target in filename:
        print(f'Filename: {filename}, Title: {title}')

Filename: S62613_1710772813836001Zn9R.pdf, Title: Text Not Found


In [39]:
slide_titles[target] = {'cleaned_title': 'Optimizing AI-Powered NPCs Cost-Efficiency Using TRT-LLM Without Sacrificing Quality by Igor Poletaev Inworld'}

In [110]:
# Validation

In [42]:
for k, v in slide_titles.items():
    if v['cleaned_title'] == 0:
        print(f"{k} || {v['cleaned_title']}")

In [41]:
import json

In [43]:
with open("pdf_full.json", "w") as f:
    json.dump(slide_titles, f)